## Homework


### Dataset

In this homework, we'll build a model for predicting if we have an image of a bee or a wasp. 
For this, we will use the "Bee or Wasp?" dataset that was obtained from [Kaggle](https://www.kaggle.com/datasets/jerzydziewierz/bee-vs-wasp) and slightly rebuilt. 

You can download the dataset for this homework from [here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip):

```bash
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
unzip data.zip
```

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch. 

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.applications.xception import Xception, preprocess_input

2023-11-13 13:24:58.810068: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 13:24:58.810121: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 13:24:58.810176: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 13:24:58.819191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



### Data Preparation

The dataset contains around 2500 images of bees and around 2100 images of wasps. 

The dataset contains separate folders for training and test sets. 


### Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).


In [2]:
def create_model():
    # Define the input layer
    inputs = keras.Input(shape=(150, 150, 3))

    # Convolutional layer
    conv_layer = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
    
    # Max pooling
    vectors = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv_layer)
    
    # Flatten
    vectors = keras.layers.Flatten()(vectors)
    
    # Dense inner layer
    inner_layer = keras.layers.Dense(60, activation='relu')(vectors)

    # Output layer
    outputs = keras.layers.Dense(1, activation='sigmoid')(inner_layer)

    # Compile the model
    optimizer = keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
    loss = keras.losses.BinaryCrossentropy()
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    return model

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

* `mean squared error`
* `binary crossentropy`
* `categorical crossentropy`
* `cosine similarity`

> **Note:** since we specify an activation for the output layer, we don't need to set `from_logits=True`



Ans: binary crossentropy



### Question 2

What's the number of parameters in the convolutional layer of our model? You can use the `summary` method for that. 

* 1 
* 65
* 896
* 11214912


In [3]:
model = create_model()
model.summary()

2023-11-13 13:25:01.707582: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-13 13:25:01.714989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-13 13:25:01.715375: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 60)                10513980  
                                                                 
 dense_1 (Dense)             (None, 1)                 61        
                                                             

### Generators and Training

For the next two questions, use the following data generator for both train and test sets:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/test directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets. 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [4]:
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory('data/train',
                                         target_size=(150,150), 
                                         batch_size=20, 
                                         shuffle=True, 
                                         class_mode='binary'
                                        )
test_ds = test_gen.flow_from_directory('data/test',
                                       target_size=(150,150), 
                                       batch_size=20, 
                                       shuffle=True,
                                       class_mode='binary'
                                      )

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [5]:
history = model.fit(train_ds, validation_data=test_ds, epochs=10)

Epoch 1/10


2023-11-13 13:25:03.693900: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-13 13:25:04.692150: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f226c2ae640 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-13 13:25:04.692178: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1060, Compute Capability 6.1
2023-11-13 13:25:04.757732: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


184/184 [==============================] - 11s 48ms/step - loss: 0.6635 - accuracy: 0.5915 - val_loss: 0.6077 - val_accuracy: 0.6503
Epoch 2/10
184/184 [==============================] - 9s 50ms/step - loss: 0.5918 - accuracy: 0.6796 - val_loss: 0.5707 - val_accuracy: 0.7092
Epoch 3/10
184/184 [==============================] - 9s 49ms/step - loss: 0.5476 - accuracy: 0.7343 - val_loss: 0.5459 - val_accuracy: 0.7266
Epoch 4/10
184/184 [==============================] - 9s 51ms/step - loss: 0.5182 - accuracy: 0.7517 - val_loss: 0.5328 - val_accuracy: 0.7418
Epoch 5/10
184/184 [==============================] - 9s 50ms/step - loss: 0.5017 - accuracy: 0.7707 - val_loss: 0.5212 - val_accuracy: 0.7538
Epoch 6/10
184/184 [==============================] - 9s 49ms/step - loss: 0.4736 - accuracy: 0.7865 - val_loss: 0.5727 - val_accuracy: 0.7070
Epoch 7/10
184/184 [==============================] - 9s 48ms/step - loss: 0.4600 - accuracy: 0.7966 - val_loss: 0.5222 - val_accuracy: 0.7516
Epoch 8/1


### Question 3

What is the median of training accuracy for all the epochs for this model?

* 0.20
* 0.40
* 0.60
* 0.80

In [6]:
np.median(history.history['accuracy'])

0.7786238789558411


### Question 4

What is the standard deviation of training loss for all the epochs for this model?

* 0.031
* 0.061
* 0.091
* 0.131


In [7]:
np.std(history.history['loss'])

0.08806620461955236

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=50,`
* `width_shift_range=0.1,`
* `height_shift_range=0.1,`
* `zoom_range=0.1,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [8]:
train_gen = ImageDataGenerator(rescale=1./255, 
                               rotation_range=50, 
                               width_shift_range=0.1, 
                               height_shift_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest'
                              )

train_ds = train_gen.flow_from_directory('data/train',
                                         target_size=(150,150), 
                                         batch_size=20, 
                                         shuffle=True, 
                                         class_mode='binary'
                                        )

Found 3677 images belonging to 2 classes.


In [9]:
history = model.fit(train_ds, validation_data=test_ds, epochs=10)

Epoch 1/10
184/184 [==============================] - 23s 126ms/step - loss: 0.5190 - accuracy: 0.7501 - val_loss: 0.4964 - val_accuracy: 0.7702
Epoch 2/10
184/184 [==============================] - 24s 128ms/step - loss: 0.4886 - accuracy: 0.7724 - val_loss: 0.5215 - val_accuracy: 0.7353
Epoch 3/10
184/184 [==============================] - 25s 133ms/step - loss: 0.4865 - accuracy: 0.7699 - val_loss: 0.4876 - val_accuracy: 0.7919
Epoch 4/10
184/184 [==============================] - 24s 132ms/step - loss: 0.4817 - accuracy: 0.7726 - val_loss: 0.4674 - val_accuracy: 0.7854
Epoch 5/10
184/184 [==============================] - 28s 151ms/step - loss: 0.4812 - accuracy: 0.7786 - val_loss: 0.5084 - val_accuracy: 0.7767
Epoch 6/10
184/184 [==============================] - 26s 143ms/step - loss: 0.4606 - accuracy: 0.7873 - val_loss: 0.4686 - val_accuracy: 0.7919
Epoch 7/10
184/184 [==============================] - 25s 136ms/step - loss: 0.4714 - accuracy: 0.7756 - val_loss: 0.5276 - val_ac

### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
> **Note:** make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.18
* 0.48
* 0.78
* 0.108


In [10]:
np.mean(history.history['val_loss'])

0.4869013547897339

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

* 0.38
* 0.58
* 0.78
* 0.98

In [11]:
np.mean(history.history['val_accuracy'][-5:])

0.7847494602203369